In [ ]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 58.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
# numpy version 1.21.5
import numpy
numpy.version.version

'1.21.5'

In [ ]:
# gensim version 4.1.2
import gensim
print(gensim.__version__)

4.1.2


In [ ]:
import numpy as np
import pandas as pd
import os
from gensim.models import Word2Vec

import json
import gensim
from nltk import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
stopwords = nltk.corpus.stopwords.words('english')

import h5py
import tqdm

import itertools
import gensim, logging
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gc
from multiprocessing import Pool
# from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from gensim.parsing.preprocessing import remove_stopwords



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def ele0(x):
    return x[0]
    
# Loads from JSON
def json_l(x):
    try:
        return json.loads(x)
    except ValueError:
        return []
        
# Preprocessing (sentence tokenizer + basik Gensim - lowercases, tokenizes, de-accents (optional))
def prepro(x):
    x = sent_tokenize(x)
    clean = []
    for j in x:
        clean.append(gensim.utils.simple_preprocess(j, min_len=1))
    return clean
    
def bigrammer(x):
    bigram = gensim.models.phrases.Phraser.load('bigram.m')
    sents = []
    for i in x:
        sents.append(bigram[i])
    return sents
    
def tfidfer(para):
    model_tfidf = gensim.models.TfidfModel.load('model_tfidf_11_2.m')
    docs_dict = Dictionary.load('docs_dict_11_2.d')
    return model_tfidf[docs_dict.doc2bow(itertools.chain(*para))] 

def preprocess_1 (data):
    data['appln_abstract_prepro'] = list(map(prepro, data.appln_abstract_st_r))
    data['appln_abstract_prepro'] = list(map(json.dumps, data.appln_abstract_prepro))
    return data
    
def preprocess_2 (data):
    data['no_json'] = list(map(json.loads, data['appln_abstract_prepro']))
    data['appln_abstract_prepro_bi'] = list(map(bigrammer, data['no_json']))
    data['appln_abstract_prepro_bi'] = list(map(json.dumps, data.appln_abstract_prepro_bi))
    return data

def preprocess_3 (data):
    data['no_json_tf'] = list(map(json.loads, data['appln_abstract_prepro_bi']))
    data['tfidf'] = list(map(tfidfer, data['no_json_tf']))
    data['tfidf'] = list(map(json.dumps, data['tfidf']))
    return data

def get_data_path(filename):
    data_dir = '/content'
    path = os.path.join(data_dir, filename)
    if data_dir != '.' and 'DEEP_QUANT_ROOT' in os.environ:
        path = os.path.join(os.environ['DEEP_QUANT_ROOT'], path)
    return path 
    
def abstract_generator_nonmult(df):
    go = 'OK'
    while go == 'OK':
        try:
            i = df['appln_abstract_prepro_bi']
        except StopIteration:
                return
        if i.isnull().sum() > 0:
            i = list(filter(lambda a: a != None, i))
            go = 'STOP'
        if i.isnull().sum() < 0:
            i = [x[0] for x in i]
        j = [json_l(x) for x in i]
        for h in j:
            try:
                yield list(itertools.chain(*h))
            except StopIteration:
                return
    

In [ ]:
data = pd.read_csv('patent_dtatset_sample_1k.csv')

In [ ]:
tokens_without_sw = [remove_stopwords(text) for text in data.appln_abstract]
data['appln_abstract_st_r'] = tokens_without_sw

In [ ]:
data_1 = preprocess_1(data)
data_1.to_csv('data.csv')

In [ ]:
sentences = Text8Corpus(get_data_path('data.csv'))

In [ ]:
bigram = Phrases(sentences, min_count=1, threshold=1, connector_words=ENGLISH_CONNECTOR_WORDS)

2022-03-04 19:52:10,588 : INFO : collecting all words and their counts
2022-03-04 19:52:10,610 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-03-04 19:52:12,004 : INFO : collected 264868 token types (unigram + bigrams) from a corpus of 402285 words and 41 sentences
2022-03-04 19:52:12,021 : INFO : merged Phrases<264868 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2022-03-04 19:52:12,027 : INFO : Phrases lifecycle event {'msg': 'built Phrases<264868 vocab, min_count=1, threshold=1, max_vocab_size=40000000> in 1.44s', 'datetime': '2022-03-04T19:52:12.027906', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [ ]:
bigram = gensim.models.phrases.Phraser(bigram)

2022-03-04 19:52:12,046 : INFO : exporting phrases from Phrases<264868 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2022-03-04 19:52:13,742 : INFO : FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<65503 phrases, min_count=1, threshold=1> from Phrases<264868 vocab, min_count=1, threshold=1, max_vocab_size=40000000> in 1.69s', 'datetime': '2022-03-04T19:52:13.742808', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [ ]:
bigram.save('bigram.m')

2022-03-04 19:52:13,776 : INFO : FrozenPhrases lifecycle event {'fname_or_handle': 'bigram.m', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-04T19:52:13.776783', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-03-04 19:52:13,828 : INFO : saved bigram.m


In [ ]:
data_2 = preprocess_2 (data_1)

2022-03-04 19:52:13,880 : INFO : loading FrozenPhrases object from bigram.m
2022-03-04 19:52:13,914 : INFO : FrozenPhrases lifecycle event {'fname': 'bigram.m', 'datetime': '2022-03-04T19:52:13.914840', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2022-03-04 19:52:13,916 : WARNING : older version of FrozenPhrases loaded without corpus_word_count
2022-03-04 19:52:13,920 : WARNING : setting corpus_word_count to 0, do not use it in your scoring function
2022-03-04 19:52:13,930 : INFO : loading FrozenPhrases object from bigram.m
2022-03-04 19:52:13,965 : INFO : FrozenPhrases lifecycle event {'fname': 'bigram.m', 'datetime': '2022-03-04T19:52:13.965033', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2022-03-04 19:52:13,978 : WARNING : older version of 

In [ ]:
data_2.to_csv('data.csv')

In [ ]:
model_with_phrases = Word2Vec(sentences=sentences, vector_size=300, window=80, min_count=1, workers=14)

2022-03-04 19:53:38,814 : INFO : collecting all words and their counts
2022-03-04 19:53:38,818 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-04 19:53:39,202 : INFO : collected 66364 word types from a corpus of 585148 raw words and 59 sentences
2022-03-04 19:53:39,206 : INFO : Creating a fresh vocabulary
2022-03-04 19:53:39,770 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 66364 unique words (100.0%% of original 66364, drops 0)', 'datetime': '2022-03-04T19:53:39.770791', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-03-04 19:53:39,778 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 585148 word corpus (100.0%% of original 585148, drops 0)', 'datetime': '2022-03-04T19:53:39.778254', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'pla

In [ ]:
model_with_phrases.save('w2v.m')

2022-03-04 19:54:15,859 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'w2v.m', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-04T19:54:15.859048', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-03-04 19:54:15,882 : INFO : storing np array 'vectors' to w2v.m.wv.vectors.npy
2022-03-04 19:54:16,023 : INFO : storing np array 'syn1neg' to w2v.m.syn1neg.npy
2022-03-04 19:54:16,287 : INFO : not storing attribute cum_table
2022-03-04 19:54:16,404 : INFO : saved w2v.m


In [ ]:
voc = model_with_phrases.wv.index_to_key

In [ ]:
# Build Gensim Dictionary
docs_dict = Dictionary([voc])
docs_dict.compactify()
docs_dict.save('docs_dict_11_2.d')

2022-03-04 19:54:16,443 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-04 19:54:17,282 : INFO : built Dictionary(66364 unique tokens: ['""\\u00ba""],', '""\\u00e0"",', '""\\u00e1"",', '""\\u00ed""],', '""\\u00ee"",']...) from 1 documents (total 66364 corpus positions)
2022-03-04 19:54:17,285 : INFO : Dictionary lifecycle event {'msg': 'built Dictionary(66364 unique tokens: [\'""\\\\u00ba""],\', \'""\\\\u00e0"",\', \'""\\\\u00e1"",\', \'""\\\\u00ed""],\', \'""\\\\u00ee"",\']...) from 1 documents (total 66364 corpus positions)', 'datetime': '2022-03-04T19:54:17.285225', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2022-03-04 19:54:17,444 : INFO : Dictionary lifecycle event {'fname_or_handle': 'docs_dict_11_2.d', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-04T19:54:17.444335', 'gensim': '4.1.2', 'pyt

In [ ]:
model_tfidf = TfidfModel((docs_dict.doc2bow(x) for x in sentences), id2word=docs_dict)
model_tfidf.save('model_tfidf_11_2.m')

2022-03-04 19:54:17,520 : INFO : collecting document frequencies
2022-03-04 19:54:17,564 : INFO : PROGRESS: processing document #0
2022-03-04 19:54:19,075 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 59 documents and 66364 features (197679 matrix non-zeros)', 'datetime': '2022-03-04T19:54:19.075828', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'initialize'}
2022-03-04 19:54:19,079 : INFO : TfidfModel lifecycle event {'fname_or_handle': 'model_tfidf_11_2.m', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-04T19:54:19.079350', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-03-04 19:54:19,521 : INFO : saved model_tfidf_11_2.m


In [ ]:
data_3 = preprocess_3 (data_2)
data_3.to_csv('data.csv')
iterator = abstract_generator_nonmult(data_3)

2022-03-04 19:54:19,569 : INFO : loading TfidfModel object from model_tfidf_11_2.m
2022-03-04 19:54:19,705 : INFO : loading id2word recursively from model_tfidf_11_2.m.id2word.* with mmap=None
2022-03-04 19:54:19,708 : INFO : TfidfModel lifecycle event {'fname': 'model_tfidf_11_2.m', 'datetime': '2022-03-04T19:54:19.708382', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2022-03-04 19:54:19,711 : INFO : loading Dictionary object from docs_dict_11_2.d
2022-03-04 19:54:19,801 : INFO : Dictionary lifecycle event {'fname': 'docs_dict_11_2.d', 'datetime': '2022-03-04T19:54:19.801253', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2022-03-04 19:54:19,819 : INFO : loading TfidfModel object from model_tfidf_11_2.m
2022-03-04 19:54:19,943 : INFO : loading id

In [ ]:
# Document matrix TF-IDF weighted
docs_vecs = (sparse2full(c, len(docs_dict)) for c in ((model_tfidf[docs_dict.doc2bow(x)] for x in iterator)))

In [ ]:
# Selected Word-Embeddings
emb_vecs_selftrained = np.vstack([model_with_phrases.wv[docs_dict.get(i)] for i in range(len(docs_dict))])
n_abstracts = len(data_2)
h5f = h5py.File('docvecs_23_4_test.h5', 'a')
dataset = h5f.create_dataset('weighted_tfidf_docvecs', (n_abstracts,300))

In [ ]:
# Generates document-vectors
pbar = tqdm.tqdm(total=n_abstracts)
start = 0
#while docs_vecs:
for i in range(1):
    a = np.vstack(next(docs_vecs) for _ in range(n_abstracts))
    pbar.update(n_abstracts)
    b = np.dot(a,emb_vecs_selftrained)
    end = start + len(b)
    dataset[start:end] = b
    start = end
pbar.close()
h5f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
100%|██████████| 1000/1000 [00:01<00:00, 571.23it/s]


In [ ]:
h5f = h5py.File('docvecs_23_4_test.h5', 'r')
dataset = h5f['weighted_tfidf_docvecs']
dataset[1]

array([-0.19839996,  0.23547788,  0.10376877,  0.33033267, -0.26161945,
       -1.385131  ,  0.56150943,  1.9125701 , -0.19289234, -1.4226279 ,
        0.8630775 , -0.8807815 , -0.6540283 ,  0.5158603 , -0.5990874 ,
       -1.2514267 ,  0.5580719 , -0.08764281,  0.55308527, -0.25173178,
       -0.14184916,  0.62122446,  0.9116332 ,  1.5644906 , -1.0611192 ,
       -0.19830024, -0.36888856,  0.45783743, -0.11906496, -0.5062977 ,
        1.5040226 , -0.21809639,  0.9584593 , -0.41485003,  1.4157261 ,
       -0.6047046 , -0.03527535, -0.39135644, -0.31362155,  0.62806344,
       -0.08497852, -0.30407113, -0.5497111 , -0.44146842, -0.24809857,
       -0.08377683, -0.17143321, -0.0159882 , -0.30042815, -0.1741448 ,
       -0.3219701 ,  0.68149227, -0.15906766,  0.4364435 ,  0.46897992,
       -1.058611  ,  0.73248506, -1.5433689 , -0.23464814, -0.48878935,
        0.4077953 ,  0.16824585,  0.288413  ,  1.0556458 , -0.18576483,
        0.6938863 , -0.17634112, -0.16597722, -1.605686  ,  0.45

In [ ]:
import scipy
scipy.spatial.distance.cosine(dataset[9],dataset[8])

0.5732941329479218

In [ ]:
# Annoy
!pip install annoy
from annoy import AnnoyIndex
t = AnnoyIndex(dataset.shape[1])  # Length of item vector that will be indexed

     |████████████████████████████████| 646 kB 5.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391670 sha256=4d3befaae0d09aba8d7c3772bfcb1b4dbc1ead29a8f45762c35fb244803d8b84
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


In [ ]:
t.on_disk_build('vecs_build.annoy')

True

In [ ]:
for i in tqdm.tqdm(range(dataset.shape[0])):
    t.add_item(i, dataset[i])

100%|██████████| 1000/1000 [00:00<00:00, 6968.71it/s]


In [ ]:
t.build(20)

True

In [ ]:
tt = AnnoyIndex(300)
tt.load('vecs_build.annoy', prefault=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


True

In [ ]:
simH5 = h5py.File('similarities_23_2.h5', 'a')

In [ ]:
dataset = simH5.create_dataset('sims_50',(10000000,3), maxshape=(None,3), compression="gzip")

In [ ]:
round(t.get_n_items()/10)

100

In [ ]:
pbar = tqdm.tqdm(total=tt.get_n_items())

blocks = []
start = 0
end = 0
counter = 0
for i in range(tt.get_n_items()):
    j = 100
    tresh = 0.0
    while tresh < 0.5:
        y = tt.get_nns_by_item(i, j, search_k=50, include_distances=True)
        tresh = y[1][-1]
        j += 50
        if j >= 500:
            break
    block = np.array(list(zip([i for _ in range(len(y[0]))],y[0],y[1])))
    end = end + len(block)
    blocks.append(block)
    counter += 1
    if counter == 1000:
        counter = 0
        blocks_np = np.vstack(blocks)
        if end >= dataset.shape[0]:
            dataset.resize((end+(end - dataset.shape[0]),3))
        dataset[start:end] = blocks_np
        blocks = []
        gc.collect()
        start = end
        pbar.update(1000)
        
        
simH5.close()

100%|██████████| 1000/1000 [00:00<00:00, 1042.33it/s]

In [ ]:
def lookup(x):
    y = tree.get_nns_by_item(x, j, search_k=5, include_distances=True)
    return np.array(list(zip([x for _ in range(len(y[0]))],y[0],y[1])))